In [ ]:
import numpy as np
import pyphi

from qutip import *
from qutip_qip.operations import *
from intrinsic_difference import intrinsic_difference
from utils import *
from compute_ces import *

In [2]:
%load_ext autoreload
%autoreload 2

# Define states and matrices

In [3]:
# Basis states (for both classical and quantum)
zero = basis(2,0)
one = basis(2,1)

# Normalized fully general quantum state
plus = (zero + one).unit()
minus = (zero - one).unit()
superposition = (1/3*zero + 2/3*one).unit()

# Density matrix representations of the states
rho_zero = zero * zero.dag() # could also be fock_dm(2,0)
rho_one = one * one.dag()
rho_plus = plus * plus.dag()
rho_minus = minus * minus.dag()
rho_sp = superposition * superposition.dag()
rho_mm = 0.5 * rho_zero + 0.5 * rho_one

rho_bell = ket2dm(bell_state(state = '00'))
rho_GHZ = ket2dm(ghz_state(N=3))
rho_W = ket2dm(w_state(N=3))
rho_bellmix = 1/3*(tensor(rho_bell, rho_zero) + tensor(rho_zero, rho_bell) + tensor(rho_zero, rho_bell).permute([1,0,2]))

rho_bennet = 1/(3-4)*(1-(tensor(rho_zero, rho_one, rho_plus) + tensor(rho_one, rho_plus, rho_zero) 
                        + tensor(rho_plus, rho_zero, rho_one) + tensor(rho_minus, rho_minus, rho_minus)))


In [4]:
rho_plus

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.5 0.5]
 [0.5 0.5]]

In [5]:
pyphi.config.MEASURE = 'ID'
pyphi.config.PARTITION_TYPE = 'ALL'
pyphi.config.PICK_SMALLEST_PURVIEW = True

In [6]:
print('ID |0> : ', intrinsic_difference(rho_zero, rho_mm))
print('ID |+> : ', intrinsic_difference(rho_plus, rho_mm))

ID |0> :  (1.0, [array([-1.+0.j, -0.+0.j])])
ID |+> :  (1.0, [array([0.70710678+0.j, 0.70710678+0.j])])


# 2 Qubits - CNOT

In [11]:
cnot()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]

## Classical results

In [7]:
print(pyphi.config)

{ 'ACTUAL_CAUSATION_MEASURE': 'PMI',
  'ASSUME_CUTS_CANNOT_CREATE_NEW_CONCEPTS': False,
  'CACHE_POTENTIAL_PURVIEWS': True,
  'CACHE_REPERTOIRES': True,
  'CACHE_SIAS': False,
  'CACHING_BACKEND': 'fs',
  'CLEAR_SUBSYSTEM_CACHES_AFTER_COMPUTING_SIA': False,
  'CUT_ONE_APPROXIMATION': False,
  'FS_CACHE_DIRECTORY': '__pyphi_cache__',
  'FS_CACHE_VERBOSITY': 0,
  'LOG_FILE': 'pyphi.log',
  'LOG_FILE_LEVEL': 'INFO',
  'LOG_STDOUT_LEVEL': 'WARNING',
  'MAXIMUM_CACHE_MEMORY_PERCENTAGE': 50,
  'MEASURE': 'ID',
  'MONGODB_CONFIG': { 'collection_name': 'cache',
                      'database_name': 'pyphi',
                      'host': 'localhost',
                      'port': 27017},
  'NUMBER_OF_CORES': -1,
  'PARALLEL_COMPLEX_EVALUATION': False,
  'PARALLEL_CONCEPT_EVALUATION': False,
  'PARALLEL_CUT_EVALUATION': True,
  'PARTITION_TYPE': 'ALL',
  'PICK_SMALLEST_PURVIEW': False,
  'PRECISION': 6,
  'PRINT_FRACTIONS': True,
  'PROGRESS_BARS': True,
  'REDIS_CACHE': False,
  'REDIS_CONFIG'

In [83]:
CNOT_tpm = cnot().data.toarray()
#CNOT_tpm = pyphi.convert.le2be_state_by_state(CNOT_tpm.real)

In [84]:
CNOT_tpm.real

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

In [85]:
# pyphi convention for state ordering is little endian
CNOT_tpm = pyphi.convert.be2le_state_by_state(CNOT_tpm.real)

In [86]:
network = pyphi.Network(CNOT_tpm)
subsystem = pyphi.Subsystem(network, (1,0))

In [14]:
ces = pyphi.compute.ces(subsystem)

In [15]:
ces

════════════════════════════════════════════════════════════
            Cause-effect structure (2 concepts)             
════════════════════════════════════════════════════════════
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
              Concept: Mechanism = [n0], φ = 1              
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
              MIC                         MIE               
  ┌──────────────────────────┐┌──────────────────────────┐  
  │  φ = 1                   ││  φ = 1                   │  
  │  Purview = [n0]          ││  Purview = [n0]          │  
  │  MIP:                    ││  MIP:                    │  
  │    n0     ∅              ││    n0     ∅              │  
  │    ─── ✕ ───             ││    ─── ✕ ───             │  
  │     ∅    n0              ││     ∅    n0              │  
  │  Repertoire:             ││  Repertoire:             │  
  │    ┌─────────────┐       ││    ┌─────────────┐       │  
  │    │ S    Pr(S)  │  

## Quantum results

In [25]:
qgate = cnot()
#qgate = tensor(qeye(2),qeye(2),qeye(2)) 

In [8]:
#rho = 0.5*tensor(rho_one, rho_one) + 0.5*tensor(rho_zero, rho_zero)
#rho = tensor(rho_one, rho_zero)
rho = rho_W

In [16]:
rho

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.33333333 0.33333333 0.         0.33333333 0.
  0.         0.        ]
 [0.         0.33333333 0.33333333 0.         0.33333333 0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.33333333 0.33333333 0.         0.33333333 0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]]

In [12]:
rho_e = evolve_unitary(rho, qgate, 'effect')
print(rho_e)

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.33333333 0.         0.33333333 0.33333333 0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.33333333 0.         0.33333333 0.33333333 0.
  0.         0.        ]
 [0.         0.33333333 0.         0.33333333 0.33333333 0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]]


In [13]:
rho_e.ptrace(0)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.66666667 0.        ]
 [0.         0.33333333]]

In [14]:
result = compute_ces(rho, qgate, direction = 'effect')

m:  (0,)
p:  (0,)  phi:  0.276692
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0.138346
p:  (0, 2)  phi:  0.138346
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0.069173
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0.276692
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0.138346
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0.138346
p:  (0, 1, 2)  phi:  0.069173
m:  (2,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (0, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (1, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0.580224
p:  (2,)  phi:  0.276692
p:  (0, 1)  phi:  0.290112
p:  (0, 2)  phi:  0.138346
p:  (1, 2)  phi:  0.628278
p:  (0, 1, 2)  phi:  0.314139
m:  (0, 1, 2)
p:  (0

In [15]:
result

[{'mech': (0,),
  'purview': (0,),
  'phi': 0.276692,
  'mip':  0     ∅ 
  ─── ✕ ───
   ∅     0 ,
  'state': [array([-1.+0.j, -0.+0.j])]},
 {'mech': (1,),
  'purview': (1,),
  'phi': 0.276692,
  'mip':  1     ∅ 
  ─── ✕ ───
   ∅     1 ,
  'state': [array([-1.+0.j, -0.+0.j])]},
 {'mech': (1, 2),
  'purview': (1, 2),
  'phi': 0.628278,
  'mip':  1     2     ∅ 
  ─── ✕ ─── ✕ ───
  1,2    ∅     ∅ ,
  'state': [array([ 0.00000000e+00+0.j, -8.50650808e-01+0.j,  2.22044605e-16+0.j,
          -5.25731112e-01-0.j])]},
 {'mech': (0, 1, 2),
  'purview': (0, 1, 2),
  'phi': 2.386639,
  'mip':  0    1,2    ∅ 
  ─── ✕ ─── ✕ ───
   0    1,2    ∅ ,
  'state': [array([ 0.        +0.j, -0.57735027-0.j,  0.        +0.j, -0.57735027+0.j,
          -0.57735027+0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j])]}]

In [41]:
result_cause = compute_ces(rho_e, qgate, direction = 'cause')

m:  (0,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0.5
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0.5
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  1.0


In [42]:
result_cause

[{'mech': (0,),
  'purview': (0, 1),
  'phi': 0.5,
  'mip':  0     ∅ 
  ─── ✕ ───
   ∅    0,1,
  'state': [array([-0.70710678-0.j,  0.        +0.j,  0.        +0.j, -0.70710678+0.j]),
   array([ 0.        +0.j, -0.70710678+0.j, -0.70710678+0.j,  0.        +0.j])]},
 {'mech': (1,),
  'purview': (0, 1),
  'phi': 0.5,
  'mip':  1     ∅ 
  ─── ✕ ───
   ∅    0,1,
  'state': [array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
   array([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j])]},
 {'mech': (0, 1),
  'purview': (0, 1),
  'phi': 1.0,
  'mip':  0     1     ∅ 
  ─── ✕ ─── ✕ ───
  0,1    ∅     ∅ ,
  'state': [array([-0.70710678-0.j,  0.        +0.j,  0.        +0.j, -0.70710678+0.j])]}]

# 3 Qubits

In [34]:
#qgate = tensor(qeye(2), cnot())
qgate = fredkin()

In [56]:
#rho = tensor(rho_plus, rho_one, rho_plus)
rho = ket2dm((ket((1,1,0)) + ket((0,0,1))).unit())
print(rho)

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.  0.  0.  0.  0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.  0.  0.  0.  0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]]


In [57]:
rho_e = evolve_unitary(rho, qgate, 'effect')
print(rho_e)

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.  0.  0.  0.5 0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.  0.  0.  0.5 0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]]


In [58]:
result = compute_ces(rho, qgate, direction = 'effect')

m:  (0,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (2,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0.438722
p:  (2,)  phi:  0.438722
p:  (0, 1)  phi:  0.219361
p:  (0, 2)  phi:  0.219361
p:  (1, 2)  phi:  0.658083
p:  (0, 1, 2)  phi:  0.329041
m:  (0, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0.438722
p:  (2,)  phi:  0.438722
p:  (0, 1)  phi:  0.219361
p:  (0, 2)  phi:  0.219361
p:  (1, 2)  phi:  0.658083
p:  (0, 1, 2)  phi:  0.329041
m:  (1, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (0, 1, 2)
p:  (0,)  phi:  1.0


In [59]:
result

[{'mech': (0, 1),
  'purview': (1, 2),
  'phi': 0.658083,
  'mip': 0,1    ∅ 
  ─── ✕ ───
   ∅    1,2,
  'state': [array([0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j])]},
 {'mech': (0, 2),
  'purview': (1, 2),
  'phi': 0.658083,
  'mip': 0,2    ∅ 
  ─── ✕ ───
   ∅    1,2,
  'state': [array([0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j])]},
 {'mech': (0, 1, 2),
  'purview': (0, 1, 2),
  'phi': 1.830075,
  'mip':  0,1     2     ∅ 
  ───── ✕ ─── ✕ ───
  0,1,2    ∅     ∅ ,
  'state': [array([ 0.        +0.j, -0.70710678-0.j,  0.        +0.j,  0.        +0.j,
           0.        +0.j, -0.70710678+0.j,  0.        +0.j,  0.        +0.j])]}]

# Measurement

In [110]:
qgate = [tensor(rho_zero, qeye(2)), tensor(rho_one, qeye(2))]
#qgate = tensor(rho_zero, qeye(2))

In [111]:
qgate

[Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
 Qobj data =
 [[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]],
 Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
 Qobj data =
 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]]]

In [119]:
rho = tensor(rho_sp, rho_mm)
#rho = rho_bell
rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.1 0.  0.2 0. ]
 [0.  0.1 0.  0.2]
 [0.2 0.  0.4 0. ]
 [0.  0.2 0.  0.4]]

In [120]:
kraus_to_super(qgate).iscptp

True

In [134]:
rho_m = evolve_cptp(rho, qgate, 'effect')
rho_m.ptrace(0)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.2 0. ]
 [0.  0.8]]

In [122]:
evolve_cptp(rho_m, qgate, 'cause')

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.1 0.  0.  0. ]
 [0.  0.1 0.  0. ]
 [0.  0.  0.4 0. ]
 [0.  0.  0.  0.4]]

In [135]:
result = compute_ces(tensor(rho_one, rho_mm), qgate, direction = 'effect')

m:  (0,)
p:  (0,)  phi:  1.0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0.5
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0


In [136]:
result

[{'mech': (0,),
  'purview': (0,),
  'phi': 1.0,
  'mip':  0     ∅ 
  ─── ✕ ───
   ∅     0 ,
  'state': [array([0.+0.j, 1.+0.j])]}]

In [103]:
result_c = compute_ces(rho_m, qgate, direction = 'cause')

m:  (0,)
p:  (0,)  phi:  0.542458
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0.271229
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0


In [104]:
result_c

[{'mech': (0,),
  'purview': (0,),
  'phi': 0.542458,
  'mip':  0     ∅ 
  ─── ✕ ───
   ∅     0 ,
  'state': [array([0.+0.j, 1.+0.j])]}]

In [132]:
rho_sp

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.2 0.4]
 [0.4 0.8]]

In [133]:
measurement_statistics(rho, qgate)

([Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
  Qobj data =
  [[0.5 0.  0.  0. ]
   [0.  0.5 0.  0. ]
   [0.  0.  0.  0. ]
   [0.  0.  0.  0. ]],
  Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
  Qobj data =
  [[0.  0.  0.  0. ]
   [0.  0.  0.  0. ]
   [0.  0.  0.5 0. ]
   [0.  0.  0.  0.5]]],
 [0.19999999999999998, 0.7999999999999999])